# Ekstraksi dan Pengolahan Data Kepemilikan Tanah

## 1. Menyiapkan coding environment, termasuk menginstall dan/atau memperbarui library

In [1]:
# Menginstall library-library
!pip install pandas
!pip install pytesseract
!pip install pdf2image
!pip install pillow
!pip install numpy
!pip install matplotlib

# Mengimport library-library untuk digunakan
import cv2
import numpy as np
import os
import pandas as pd
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
from matplotlib import pyplot as plt

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Loading the preprocessed template CSV file
template = pd.read_csv('DATA C-DESA NOBOREJO preprocessed.csv')

#### !!! TESTING "berkas 500-520 EXAMPLE 2.pdf !!!

##### PDF to PNG conversion

In [3]:
# Initialize a new empty dataframe with the same columns as the template
df = pd.DataFrame(columns=template.columns)
df.head()

# Save CSV to the output directory
output_dir = 'Berkas Files/images of berkas 500-520 EXAMPLE 2'
output_file = 'output.csv'
df.to_csv(os.path.join(output_dir, output_file), index=False)

In [4]:
# Create subdirectory to hold images inside the Berkas Files subdirectory
output_folder = 'Berkas Files/images of berkas 500-520 EXAMPLE 2'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Convert PDF to images
pdf_path = 'Berkas Files/berkas 500-520 EXAMPLE 2.pdf'
images = convert_from_path(pdf_path)

# Save each image to the output folder
for i, image in enumerate(images):
    image_path = os.path.join(output_folder, f'halaman_{i+1}.png')
    image.save(image_path, 'PNG')

# Display the paths of the saved images
image_paths = [os.path.join(output_folder, f'halaman_{i+1}.png') for i in range(len(images))]
print("Images saved at:")
for path in image_paths:
    print(path)

Images saved at:
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_1.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_2.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_3.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_4.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_5.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_6.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_7.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_8.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_9.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_10.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_11.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_12.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_13.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_14.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_15.png
Berkas Files/images of berkas 500-520 EXAMPLE 2

##### Image preprocessing

In [5]:
# Load the image
image_path = 'Berkas Files/images of berkas 500-520 EXAMPLE 2/halaman_1.png'

# Read the image using OpenCV
image = cv2.imread(image_path, cv2.IMREAD_COLOR)

# Create output subdirectory
output_dir = 'Berkas Files/images of berkas 500-520 EXAMPLE 2/Preprocessed halaman 1 stuff'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [6]:
# Step 1: Apply thresholding - to separate the text from the background
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale (just in case)
_, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

# Save the output for this step
step_1_path = os.path.join(output_dir, 'Step 1 - Thresholded Image.png')
cv2.imwrite(step_1_path, binary)

print(f"Step 1 completed. Thresholded image saved at: {step_1_path}")

Step 1 completed. Thresholded image saved at: Berkas Files/images of berkas 500-520 EXAMPLE 2/Preprocessed halaman 1 stuff\Step 1 - Thresholded Image.png


In [7]:
# # Step 2: Noise removal using Median Blur - to be used only if the image is noisy
# denoised = cv2.medianBlur(binary, 3)

# Save the output for this step
# step_2_path = os.path.join(output_dir, 'Step 2 - Denoised Image.png')
# cv2.imwrite(step_2_path, denoised)

# print(f"Step 2 completed. Denoised image saved at: {step_2_path}")

# SKIPPING THIS STEP FOR BETTER QUALITY SCANNED IMAGES

In [9]:
# Step 3: Table Detection using Contours
try:
    # Define kernel for morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))

    # Detect edges using Canny
    edges = cv2.Canny(binary, 50, 150, apertureSize=3)

    # Dilate edges to strengthen table lines
    dilated = cv2.dilate(edges, kernel, iterations=2)

    # Find contours
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw bounding boxes around detected table regions
    table_detected = binary.copy()
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        # Filter based on size (adjust thresholds as needed)
        if w > 50 and h > 20:  # Minimum width and height
            cv2.rectangle(table_detected, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Save the output for table detection
    step_3_path = os.path.join(output_dir, 'Step 3 - Table Detection.png')
    cv2.imwrite(step_3_path, table_detected)

    print(f"Step 3 completed. Table detection image saved at: {step_3_path}")
except Exception as e:
    print(f"An error occurred during table detection: {e}")

Step 3 completed. Table detection image saved at: Berkas Files/images of berkas 500-520 EXAMPLE 2/Preprocessed halaman 1 stuff\Step 3 - Table Detection.png


## 3. Memproses teks yang diekstrak

## 4. Save ke dalam file Excel (file format '.csv')